In [2]:
#imports!
import datetime as dt
from pmaw import PushshiftAPI
import pandas as pd

In [21]:
#epoch time objects
start_time = int(dt.datetime(2013, 1, 1).timestamp())
end_time = int(dt.datetime(2023, 1, 1).timestamp())
print(start_time, end_time)

1357016400 1672549200


In [22]:
#instantiate the pushshift wrapper
api = PushshiftAPI()
posts = api.search_submissions(subreddit="aiethics",  since = start_time, until=end_time, safe_exit=True)

In [23]:
len(posts)

517

In [24]:
res = [r for r in posts]

In [25]:
fin_data = pd.json_normalize(res)

In [26]:
fin_data = fin_data[['author', 'created_utc', 'id', 'url', 'title', 'selftext', 'score', 'num_comments']]

In [27]:
fin_data.shape

(517, 8)

In [28]:
fin_data = fin_data[~(fin_data['selftext'] == '[deleted]') & ~(fin_data['selftext'] == '[removed]') & ~(fin_data['selftext'] == '') & ~(fin_data['author'] == '[deleted]')] 

In [29]:
fin_data.shape

(94, 8)

In [17]:
fin_data = fin_data.dropna(subset=['selftext'])

In [18]:
fin_data.shape

(240, 8)

In [30]:
fin_data.to_csv("aiethics_data_pmaw_posts_cleaned.csv")

# in this part we will collect all the column data

In [31]:
subreddit_ids = fin_data['id'].to_list()

In [32]:
len(subreddit_ids)

94

In [38]:
#authenticate praw to collect comments!
import praw
import pandas as pd

reddit = praw.Reddit(
    client_id="lr0n5LZ0t2lA_g",
    client_secret="zxFWs7TlupQmD9War86reJJXoQs",
    user_agent="Mozilla/5.0",
    username="sree2016",
    password="Tokio_1989",
)


In [39]:
final_data = {'author':[], 'created_utc':[], 'sub_id':[],'id':[], 'parent_id':[], 'score':[], 'selftext':[]}
counter = 1
for sub_id in subreddit_ids:
    submission = reddit.submission(id=sub_id)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        if comment.author != '':
            final_data['author'].append(comment.author)
        else:
            final_data['author'].append('None')
        final_data['created_utc'].append(comment.created_utc)
        final_data['sub_id'].append(sub_id)
        final_data['parent_id'].append(comment.parent_id)
        final_data['id'].append(comment.id)
        final_data['score'].append(comment.score)
        try:
            if comment.body != '':
                final_data['selftext'].append(comment.body)
            else:
                final_data['selftext'].append('None')
        except:
            final_data['selftext'].append('No-Text')
         
    print(f'{counter}, Currently accessing comments for', sub_id)
    counter+=1

1, Currently accessing comments for 7hft48
2, Currently accessing comments for 7em793
3, Currently accessing comments for 72iszw
4, Currently accessing comments for 7197l1
5, Currently accessing comments for 6xg682
6, Currently accessing comments for 6wyl36
7, Currently accessing comments for 6sq5sf
8, Currently accessing comments for 6kg7u9
9, Currently accessing comments for 662yix
10, Currently accessing comments for aaj367
11, Currently accessing comments for a8n2dv
12, Currently accessing comments for a4p7eg
13, Currently accessing comments for 9sfz6f
14, Currently accessing comments for 8w67km
15, Currently accessing comments for 8t368d
16, Currently accessing comments for 8lbev2
17, Currently accessing comments for 8aalz7
18, Currently accessing comments for 7u1ag1
19, Currently accessing comments for 7ohuvl
20, Currently accessing comments for 5ju0g7
21, Currently accessing comments for 5fohd4
22, Currently accessing comments for 57vno0
23, Currently accessing comments for 55hb

In [40]:
final_data = pd.DataFrame(final_data)

In [42]:
final_data.shape

(266, 7)

In [1]:
final_data.to_csv("aiethics_comments.csv", quoting=csv.QUOTE_NONE)

NameError: name 'final_data' is not defined